In [1]:
import os
import sys
import importlib
import numpy as np
script_dir = '/mnt/c/Users/kevin/OneDrive/2-code/1-Research_projects/1-project-categorical-MMN/notebooks'
src_dir = os.path.join(script_dir, '..', 'src')
sys.path.append(src_dir)
import parameters as params
import classes as cl
import basic_functions as bf
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

This file will study the data from script 3: huge set of parameters for prediciton index

In [3]:
data_dir = os.path.join(script_dir,'..', 'data/sims/script3') 
from itertools import product
import json

def create_list_dict(d):
    ''' This funciton returns all the possible combinations of elements within the keys
    '''
    list_dict = []
    key_list = list(d.keys())
    length_list = [len(d[key]) for key in key_list]
    nbr_list = len(key_list)
    
    all_lists = list(product(*[d[key] for key in key_list]))
    return all_lists

dict_param = {
    'adaptation_list' : [-0.01,-0.02,-0.03,-0.04,-0.05,0.0],
'top_down_feedback_list' : [0.0,0.5,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0],
'Tinter_list' : [0.2, 0.5,0.7,1.0,2.0],
'dev_id_list' : [88,78,68,58,48,38,28],
'delay_list': [50.0,100.0]    
}
    
all_dicts = create_list_dict(dict_param)

# list_param = all_dicts[2]

df = pd.DataFrame(columns=['adaptation','top_down_feedback','Tinter','dev_id','delay', 'multi_control', 'proba_control', 'proba_dev'])

for i in range(len(all_dicts)):
    #print(i)
    list_param = all_dicts[i]    

    name = 'script3_mean_firing_rates_dev_id'+str(list_param[3])+'adapt_'+str(list_param[0])+'topdown_'+str(list_param[1])+'Tinter_'+str(list_param[2])+'delay_'+str(list_param[4])+'.json'
    file_dir = os.path.join(data_dir, name)
    index_temp = json.load(open(file_dir))
    
    adapt = list_param[0]
    top_down = list_param[1]
    Tinter = list_param[2]
    dev_id = list_param[3]
    delay = list_param[4]
    
    multi_control = index_temp['multi_control']
    proba_control = index_temp['proba_control']
    proba_dev = index_temp['proba_dev']
    
    df.loc[i] = [adapt, top_down, Tinter, dev_id, delay, multi_control, proba_control, proba_dev]
    
    
df['MMN'] = df['proba_dev'] - df['proba_control']
df['MMN_norm'] = df['MMN']/df['proba_control']
df['index_pred'] = (df['proba_dev'] - df['multi_control'])/(df['proba_dev'] - df['proba_control'])
df['index_adapt'] = (df['proba_control'] - df['multi_control'])/(df['proba_dev'] - df['proba_control'])
    


In [17]:
import sqlite3
name = 'script3_MMN_effect_database.db'
file_dir = os.path.join(data_dir, name)
con = sqlite3.connect(file_dir)
c = con.cursor()
c.execute("""Create table if not exists table_script3 (dummy_name text,
          adaptation float, 
          top_down_feedback float
          , Tinter float
          , dev_id float
          , delay float
          , multi_control float
          , proba_std float
          , proba_dev float
          , MMN float
          , MMN_norm float
          , idx_pred float)""")
con.commit()
con.close()
for i in range(len(df['MMN'])):
    con = sqlite3.connect(file_dir)
    name = str(i)
    c = con.cursor()
    c.execute("""INSERT INTO table_script3 ( dummy_name,
            adaptation , 
            top_down_feedback 
            , Tinter 
            , dev_id 
            , delay 
            , multi_control 
            , proba_std 
            , proba_dev 
            , MMN 
            , MMN_norm 
            , idx_pred )""" """VALUES (?,?,?,?,?,?,?,?,?,?,?,?)""", (name,df['adaptation'][i], df['top_down_feedback'][i], df['Tinter'][i], df['dev_id'][i], df['delay'][i], df['multi_control'][i], df['proba_control'][i], df['proba_dev'][i], df['MMN'][i], df['MMN_norm'][i], df['index_pred'][i]))
    con.commit()
    con.close()


In [16]:
con.close()

In [4]:
tableau_dir = os.path.join(script_dir,'..', 'data/sims/tableau') 

name = "script3.csv"
file_dir = os.path.join(tableau_dir, name)
df.to_csv(file_dir)

In [14]:
import plotly.express as px
import plotly.graph_objects as go

df_temp = df.query('Tinter == 0.5 & delay == 50.0 & dev_id == 68')

df_temp = df_temp[['adaptation','top_down_feedback','multi_control']].copy()

# fig = go.Figure(data = go.Heatmap(z = df_temp['multi_control'], x = df_temp['adaptation'], y = df_temp['top_down_feedback']))
# fig.show()

In [15]:
df_temp = df.query(' delay == 50.0 & dev_id > 40')
df_temp = df_temp[['adaptation','top_down_feedback','multi_control','proba_dev','proba_control', 'dev_id','Tinter']].copy()

fig = px.line(df_temp, x = 'top_down_feedback', y = 'multi_control', color = 'adaptation', markers = True)
fig.update_layout(title = 'Firing rate in multi_control paradigm at Tinter = 0.5 and delay = 50.0 and dev_id = 68')
fig.show()

In [16]:
fig = px.line(df_temp, x = 'top_down_feedback', y = 'proba_control', color = 'adaptation', markers = True)
fig.update_layout(title = 'Firing rate in proba_control paradigm at Tinter = 0.5 and delay = 50.0 and dev_id = 68')
fig.show()

In [17]:
fig = px.line(df_temp, x = 'top_down_feedback', y = 'proba_dev', color = 'adaptation', markers = True)
fig.update_layout(title = 'Firing rate in proba_dev paradigm at Tinter = 0.5 and delay = 50.0 and dev_id = 68')
fig.show()

In [25]:
df_temp['MMN'] = (df_temp.proba_dev - df_temp.proba_control)#/(df_temp.proba_dev - df_temp.proba_control)
fig = px.line(df_temp, x = 'top_down_feedback', y = 'MMN', color = 'adaptation', markers = True, facet_row = 'dev_id', facet_col = 'Tinter')
fig.update_layout(title = 'MMN at delay = 50.0')
fig.show()

# TODO a figure of just the weights through learning time
# TODO do I want simulation with different Hebbian rules strength?
#TODO summarize results, new simulaitons and so on



In [21]:
df_temp['index_pred'] = (df_temp.proba_dev - df_temp.multi_control)/(df_temp.proba_dev - df_temp.proba_control)
df_temp['MMN_relative'] = (df_temp.proba_dev-df_temp.proba_control)/(df_temp.proba_control)
fig = px.line(df_temp, x = 'top_down_feedback', y = 'MMN_relative', color = 'adaptation', markers = True, facet_row = 'dev_id', facet_col = 'Tinter')
fig.update_layout(title = 'MMN at Tinter = 0.5 and delay = 50.0')
fig.show()



In [24]:

df_temp_q = df_temp.query('MMN_relative> 0.05 & Tinter > 0.2')
fig = px.line(df_temp_q, x = 'top_down_feedback', y = 'index_pred', color = 'adaptation', markers = True, facet_row = 'dev_id', facet_col = 'Tinter')
fig.update_layout(title = 'Index pred at Tinter = 0.5 and delay = 50.0 ')
fig.show()

In [32]:
df_temp_q = df_temp.query('MMN > 1.0 & Tinter == 0.5 & dev_id == 58')
fig = px.line(df_temp_q, x = 'top_down_feedback', y = 'index_pred', color = 'adaptation', markers = True)
fig.show()

In [73]:
# TODO fit by a linear regression and plot the results

from sklearn.linear_model import LinearRegression

fig = px.line(df_temp_q, x = 'top_down_feedback', y = 'MMN', color = 'adaptation', markers = True)


for i in [-0.01,-0.02,-0.03,-0.04]:
    temp = df_temp_q[df_temp_q['adaptation'] == i]
    l = np.array(temp['top_down_feedback'])
    l = l[:,np.newaxis]
    #l = np.reshape(l,(len(l),1))
    m = np.array(temp['MMN'])
    m = m[:,np.newaxis]
    # print(l)
    reg = LinearRegression().fit(l,m)
    # df_temp_2 = pd.DataFrame(list(zip(temp['top_down_feedback'],reg.predict(l))), columns = ['top_down','pred'])
    # figtemp = px.line(df_temp_2, x='top_down', y='pred', color = 'test')
    fig.add_trace(go.Scatter(x = temp['top_down_feedback'], y = np.reshape(reg.predict(l),-1), mode = 'lines', name = str(i)))
fig.show()

In [75]:

fig = px.line(df_temp_q, x = 'top_down_feedback', y = 'index_pred', color = 'adaptation', markers = True)


for i in [-0.01,-0.02,-0.03,-0.04]:
    temp = df_temp_q[df_temp_q['adaptation'] == i]
    l = np.array(temp['top_down_feedback'])
    l = l[:,np.newaxis]
    #l = np.reshape(l,(len(l),1))
    m = np.array(temp['index_pred'])
    m = m[:,np.newaxis]
    # print(l)
    reg = LinearRegression().fit(l,m)
    # df_temp_2 = pd.DataFrame(list(zip(temp['top_down_feedback'],reg.predict(l))), columns = ['top_down','pred'])
    # figtemp = px.line(df_temp_2, x='top_down', y='pred', color = 'test')
    fig.add_trace(go.Scatter(x = temp['top_down_feedback'], y = np.reshape(reg.predict(l),-1), mode = 'lines', name = str(i)))
fig.show()

In [86]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df_temp['top_down_feedback'], y=df_temp['MMN'], name='proba_dev', color = df_temp['adaptation'], markers = True), row=1, col=1)
fig.add_trace(go.Scatter(x=df_temp['top_down_feedback'], y=df_temp['index_pred'], name='proba_control', color = 'adaptation', markers = True), row=2, col=1)
fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'color'

Did you mean "fill"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        dx
            Sets the x coordinate ste

In [83]:
df_temp['index_adapt'] = np.abs((df_temp.proba_control - df_temp.multi_control)/(df_temp.proba_dev - df_temp.proba_control))
fig = px.line(df_temp, x = 'top_down_feedback', y = 'index_adapt', color = 'adaptation', markers = True)
fig.update_layout(title = 'Index pred at Tinter = 0.5 and delay = 50.0 and dev_id = 68')
fig.show()

In [76]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data

df_temp = df.query('delay == 50.0 & dev_id == 68')

df_temp = df_temp[['adaptation','top_down_feedback','multi_control','Tinter']].copy()



# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Influence Adaptation"),
    dcc.Checklist(
        id="checklist",
        options=[{"label": 'Tinter ' + str(x) + 's', "value": x} 
                 for x in dict_param['Tinter_list']],
        value=dict_param['Tinter_list'],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id='line-chart'),
    # html.Label([
    #     "colorscale",
    #     dcc.Dropdown(
    #         id='colorscale-dropdown', clearable=False,
    #         value='plasma', options=[
    #             {'label': c, 'value': c}
    #             for c in px.colors.named_colorscales()
    #         ])
    # ]),
    
])
# Define callback to update graph
@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(Tinterlist):
    mask = df_temp.Tinter.isin(Tinterlist)
    fig = px.scatter(df_temp[mask], 
        x="top_down_feedback", y="multi_control", color='adaptation', render_mode = "webgl", symbol = 'Tinter', size = 'Tinter')
    return fig

# @app.callback(
#     Output('graph', 'figure'),
#     [Input("colorscale-dropdown", "value")]
# )
# def update_figure(colorscale):
#     return px.line(
#         df_temp, x="top_down_feedback", y="multi_control", color="adaptation",
#         color_scale=colorscale, markers = True, 
#         render_mode="webgl", title="Multi-control paradigm"
#     )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

/home/kevin/.local/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [26]:
m = df_temp.Tinter.isin([0.2])
dftt = df_temp[m]
dftt.head()

,adaptation,top_down_feedback,multi_control,Tinter
2,-0.01,0.0,26.627871,0.2
52,-0.01,0.5,25.410654,0.2
102,-0.01,1.0,22.375813,0.2
152,-0.01,1.1,21.588060,0.2
